In [5]:
# 05_Kriging_Interpolation_LocalGuideline.ipynb
# Outputs two figures per day:
#   (A) Continuous heatmap (RdBu_r), fixed 0–100 scale + guideline contours
#   (B) Categorical map using local PM2.5 categories (green→maroon)
# Also writes a GeoTIFF with the kriged grid per day.

import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import rasterio
from rasterio.transform import from_origin
from shapely.ops import unary_union
from fiona.env import Env
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# -------- Paths
DATA_DIR   = r"C:\Users\krish\Desktop\SpatialCARE\DailyGPKG"
PASIG_SHP  = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_R_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\rasters"
OUT_F_DIR  = r"C:\Users\krish\Desktop\SpatialCARE\Outputs\figures\kriging_local"
os.makedirs(OUT_R_DIR, exist_ok=True)
os.makedirs(OUT_F_DIR, exist_ok=True)

# -------- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7, 7), 150

# -------- Grid/Kriging
GRID_RES_M = 100
VARIOGRAM_MODEL = "spherical"
RETRY_NUGGET = 1e-3
COVERAGE_MAX_DIST_M = 3000

# -------- Continuous (analysis) style — red high, blue low
CMAP_CONT = "RdBu_r"                         # valid colormap name
VMIN, VMAX = 0.0, 100.0                      # fixed across all days
NORM_CONT = mcolors.PowerNorm(gamma=0.6, vmin=VMIN, vmax=VMAX)
GUIDE_LEVELS = [25, 35, 45, 55, 90]          # thin contours at these

# -------- Local guideline bins + colors (categorical/policy)
BINS    = [0.0, 25.0, 35.0, 45.0, 55.0, 90.0, np.inf]
LABELS  = [
    "Good (0–25.0)",
    "Fair (25.1–35.0)",
    "Unhealthy (sensitive) (35.1–45.0)",
    "Very unhealthy (45.1–55.0)",
    "Acutely unhealthy (55.1–90.0)",
    "Emergency (≥91)"
]
COLORS  = ["#00E400", "#FFFF00", "#FF7E00", "#FF0000", "#8F3F97", "#7E0023"]
CMAP_CAT = mpl.colors.ListedColormap(COLORS)
NORM_CAT = mpl.colors.BoundaryNorm(BINS, ncolors=len(COLORS), clip=False)

def pick_pm_col(cols):
    for c in cols:
        if str(c).lower() in ("pm25", "pm_25", "pm2_5", "pm2.5"):
            return c
    return None

# -------- Boundary & grid
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

minx, miny, maxx, maxy = geom.bounds
xs = np.arange(minx, maxx + GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy + GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)
transform = from_origin(xs.min(), ys.max(), GRID_RES_M, GRID_RES_M)

# -------- Iterate daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    day  = base.replace("date_", "")

    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude", "lon", "x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude", "lat", "y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print("Skip (no geometry):", base)
            continue

    pm_col = pick_pm_col(g.columns)
    if pm_col is None:
        print("Skip (no PM2.5):", base)
        continue

    # Points in UTM, de-duplicate exact XY, clip negatives
    pts = g.to_crs(UTM51)[[pm_col, "geometry"]].copy()
    pts["x"] = pts.geometry.x
    pts["y"] = pts.geometry.y
    pts = pts.groupby(["x", "y"], as_index=False)[pm_col].mean()
    pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))

    x = pts["x"].to_numpy()
    y = pts["y"].to_numpy()
    z = np.clip(pts[pm_col].astype(float).to_numpy(), 0, None)
    if len(z) < 3:
        print("Skip (n<3):", base)
        continue

    # --- Kriging (retry with a nugget if singular)
    try:
        OK = OrdinaryKriging(
            x, y, z, variogram_model=VARIOGRAM_MODEL,
            enable_plotting=False, verbose=False, coordinates_type="euclidean"
        )
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zg = np.array(zg)
    except LinAlgError:
        OK = OrdinaryKriging(
            x, y, z, variogram_model=VARIOGRAM_MODEL, nugget=RETRY_NUGGET,
            enable_plotting=False, verbose=False, coordinates_type="euclidean"
        )
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zg = np.array(zg)

    # --- Coverage + boundary masks
    tree = cKDTree(np.c_[x, y])
    dists, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
    cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)

    centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
    inside = centers.within(geom).to_numpy().reshape(zg.shape)

    valid = cover & inside
    zmask = np.where(valid, np.maximum(zg, 0.0), np.nan).astype("float32")
    if not np.isfinite(zmask).any():
        print("Skip (no valid cells):", base)
        continue

    # --- Save GeoTIFF
    tif = os.path.join(OUT_R_DIR, f"{base}_pm25_kriged.tif")
    with rasterio.open(
        tif, "w", driver="GTiff",
        height=zmask.shape[0], width=zmask.shape[1],
        count=1, dtype="float32", crs=UTM51, transform=transform, nodata=np.nan
    ) as dst:
        dst.write(zmask, 1)

    # === (A) Continuous map (RdBu_r) — fixed 0–100, with guideline contours
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    im = ax.imshow(
        zmask,
        extent=[xs[0], xs[-1], ys[0], ys[-1]],
        origin="lower",
        cmap=CMAP_CONT,
        norm=NORM_CONT
    )
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)  # <- ax passed by keyword

    # stations colored by value (same cmap/norm), size scaled by value
    s = 30 + 170 * (z / max(1e-6, z.max()))
    ax.scatter(x, y, c=z, s=s, cmap=CMAP_CONT, norm=NORM_CONT,
               edgecolors="white", linewidths=0.8, zorder=3)

    # mark max station
    i_max = int(np.nanargmax(z))
    ax.scatter([x[i_max]], [y[i_max]], marker="*", s=300, c=[z[i_max]],
               cmap=CMAP_CONT, norm=NORM_CONT, edgecolors="black",
               linewidths=0.9, zorder=4)

    # guideline contours
    try:
        cs = ax.contour(grid_x, grid_y, zmask, levels=GUIDE_LEVELS,
                        colors="black", linewidths=0.8, linestyles="--")
        ax.clabel(cs, fmt=lambda v: f"{int(v)} µg/m³", inline=True, fontsize=8)
    except Exception:
        pass

    ax.set_title(f"PM₂.₅ Kriging — {day} (continuous, 0–100 µg/m³)")
    ax.set_axis_off()
    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=NORM_CONT, cmap=CMAP_CONT), ax=ax, shrink=0.8)
    cb.set_label("PM₂.₅ (µg/m³)")
    cb.set_ticks([0, 25, 35, 45, 55, 90, 100])

    outA = os.path.join(OUT_F_DIR, f"{base}_kriged_continuous.png")
    plt.tight_layout(); plt.savefig(outA, bbox_inches="tight"); plt.close(fig)

    # === (B) Categorical map (local guideline colors)
    idx = np.digitize(zmask, BINS, right=True) - 1  # -1..5 (NaN where zmask NaN)
    cat_mask = np.ma.masked_where(~np.isfinite(zmask), idx)

    fig2, ax2 = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    im2 = ax2.imshow(
        cat_mask,
        extent=[xs[0], xs[-1], ys[0], ys[-1]],
        origin="lower",
        cmap=CMAP_CAT,
        norm=NORM_CAT
    )
    pasig_utm.boundary.plot(ax=ax2, color="black", linewidth=0.8)  # <- ax passed by keyword
    ax2.set_title(f"PM₂.₅ Kriging — {day} (local guideline)")
    ax2.set_axis_off()

    # category legend
    patches = [mpatches.Patch(color=c, label=l) for c, l in zip(COLORS, LABELS)]
    ax2.legend(handles=patches, loc="upper left", bbox_to_anchor=(1.01, 1.0),
               fontsize=9, frameon=True)

    outB = os.path.join(OUT_F_DIR, f"{base}_kriged_categorical.png")
    plt.tight_layout(); plt.savefig(outB, bbox_inches="tight"); plt.close(fig2)

    print("Saved:", os.path.basename(outA), "|", os.path.basename(outB))

Saved: date_2025-02-06_kriged_continuous.png | date_2025-02-06_kriged_categorical.png
Saved: date_2025-02-07_kriged_continuous.png | date_2025-02-07_kriged_categorical.png
Saved: date_2025-02-08_kriged_continuous.png | date_2025-02-08_kriged_categorical.png
Saved: date_2025-02-09_kriged_continuous.png | date_2025-02-09_kriged_categorical.png
Saved: date_2025-02-10_kriged_continuous.png | date_2025-02-10_kriged_categorical.png
Saved: date_2025-02-11_kriged_continuous.png | date_2025-02-11_kriged_categorical.png
Saved: date_2025-02-12_kriged_continuous.png | date_2025-02-12_kriged_categorical.png
Saved: date_2025-02-13_kriged_continuous.png | date_2025-02-13_kriged_categorical.png
Saved: date_2025-02-14_kriged_continuous.png | date_2025-02-14_kriged_categorical.png
Saved: date_2025-02-15_kriged_continuous.png | date_2025-02-15_kriged_categorical.png
Saved: date_2025-02-16_kriged_continuous.png | date_2025-02-16_kriged_categorical.png
Saved: date_2025-02-17_kriged_continuous.png | date_20